In [7]:
import re
import csv
from pathlib import Path
import polars as pl
from datasets import load_dataset

## Load metadata dataset

## [Dataset description](https://huggingface.co/datasets/arxiv-community/arxiv_dataset)

In [8]:
dataset = load_dataset("arxiv_dataset", data_dir='./data', split='train', verification_mode='no_checks')
df = dataset.to_polars()
df.head()

id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,update_date
str,str,str,str,str,str,str,str,str,str,str,str
"""0704.0001""","""Pavel Nadolsky""","""C. Bal\'azs, E. L. Berger, P. …","""Calculation of prompt diphoton…","""37 pages, 15 figures; publishe…","""Phys.Rev.D76:013009,2007""","""10.1103/PhysRevD.76.013009""","""ANL-HEP-PR-07-12""","""hep-ph""",null,""" A fully differential calcula…","""2008-11-26"""
"""0704.0002""","""Louis Theran""","""Ileana Streinu and Louis Thera…","""Sparsity-certifying Graph Deco…","""To appear in Graphs and Combin…",null,null,null,"""math.CO cs.CG""","""http://arxiv.org/licenses/none…",""" We describe a new algorithm,…","""2008-12-13"""
"""0704.0003""","""Hongjun Pan""","""Hongjun Pan""","""The evolution of the Earth-Moo…","""23 pages, 3 figures""",null,null,null,"""physics.gen-ph""",null,""" The evolution of Earth-Moon …","""2008-01-13"""
"""0704.0004""","""David Callan""","""David Callan""","""A determinant of Stirling cycl…","""11 pages""",null,null,null,"""math.CO""",null,""" We show that a determinant o…","""2007-05-23"""
"""0704.0005""","""Alberto Torchinsky""","""Wael Abu-Shammala and Alberto …","""From dyadic $\Lambda_{\alpha}$…",null,"""Illinois J. Math. 52 (2008) no…",null,null,"""math.CA math.FA""",null,""" In this paper we show how to…","""2013-10-15"""


## Filter to PDF collection and append first page of text

In [15]:
pdf_files = list(Path("./data/pdf").glob("*.pdf"))
pdf_files.sort()
print(pdf_files)

# text_files = list(Path("./data/text").glob("*.txt"))
# text_files.sort()

# id_pattern = re.compile(r"(.+)v\d+\.[pdf|txt]")
# ids = [
#     id_pattern.match(pdf.name).group(1) for pdf in pdf_files
# ]

# df_sample = df[df['id'].isin(ids)].copy()

# def get_pdf_path(id):
#     for pdf in pdf_files:
#         if id in str(pdf.stem):
#             return pdf.name
#     return None

# def get_text(id):
#     for text in text_files:
#         if id in str(text.stem):
#             return text.read_text()

# df_sample['src_pdf'] = df_sample.apply(lambda row: get_pdf_path(row['id']), axis=1)
# df_sample['text'] = df_sample.apply(lambda row: get_text(row['id']), axis=1)
# df_sample.dropna(subset = ['src_pdf'], inplace=True)
# df_sample.to_csv("./data/arxiv_metadata.csv", index=False, quoting=csv.QUOTE_ALL)
# df_sample.to_parquet("./data/arxiv_metadata.parquet", index=False)
# print(f"number of rows: {len(df_sample)}")
# df_sample.head()


[]
